In [1]:
"""
Signal Evaluation and Labeling

This script evaluates pre-generated trading signals from gold_train.csv and labels them as
'Successful', 'Unsuccessful', or 'Neutral' based on subsequent price movements.
"""

"\nSignal Evaluation and Labeling\n\nThis script evaluates pre-generated trading signals from gold_train.csv and labels them as\n'Successful', 'Unsuccessful', or 'Neutral' based on subsequent price movements.\n"

In [2]:
import pandas as pd
import numpy as np

In [3]:
def main():
    # Define file paths
    input_csv = './data/gold_train.csv'
    output_csv = './gold_train_labeled.csv'

    # Load the dataset
    try:
        df = pd.read_csv(input_csv)
        print(f"Loaded {len(df)} rows from {input_csv}")
    except FileNotFoundError:
        print(f"Error: Input file '{input_csv}' not found.")
        return
    except Exception as e:
        print(f"Error loading CSV {input_csv}: {e}")
        return

    # Parse datetime and sort
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values(by='datetime').reset_index(drop=True)

    # Display first few rows
    print("\nFirst 5 rows of the dataset:")
    print(df.head())

    # Verify required columns
    required_cols = ['close', 'Signal']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"Error: Missing required columns: {missing_cols}")
        return

    # Calculate price changes
    df['next_close'] = df['close'].shift(-1)
    df['price_diff'] = df['next_close'] - df['close']

    # Initialize strategy_success column
    df['strategy_success'] = np.nan

    # Evaluate BUY signals (Signal == 1)
    buy_mask = df['Signal'] == 1
    df.loc[buy_mask, 'strategy_success'] = df.loc[buy_mask, 'price_diff'] > 0

    # Evaluate SELL signals (Signal == -1)
    sell_mask = df['Signal'] == -1
    df.loc[sell_mask, 'strategy_success'] = df.loc[sell_mask, 'price_diff'] < 0

    # Mark HOLD signals (Signal == 0) as Neutral
    hold_mask = df['Signal'] == 0
    df.loc[hold_mask, 'strategy_success'] = pd.NA

    # Handle the last row (no next price)
    last_row_mask = df['next_close'].isna()
    df.loc[last_row_mask, 'strategy_success'] = pd.NA

    # Convert to readable labels
    success_map = {True: 'Successful', False: 'Unsuccessful', pd.NA: 'Neutral'}
    df['strategy_success'] = df['strategy_success'].map(success_map).fillna('Neutral')

    # Display results
    print("\nSignal Success Distribution:")
    print(df['strategy_success'].value_counts())

    print("\nExample rows with strategy evaluation:")
    print(df[['datetime', 'close', 'Signal', 'next_close', 'price_diff', 'strategy_success']].head(10))

    # Calculate success rates
    buy_signals = df[df['Signal'] == 1]
    sell_signals = df[df['Signal'] == -1]
    hold_signals = df[df['Signal'] == 0]

    print("\nSignal Distribution:")
    print(f"Total signals: {len(df)}")
    print(f"BUY signals: {len(buy_signals)}")
    print(f"SELL signals: {len(sell_signals)}")
    print(f"HOLD signals: {len(hold_signals)}")

    print("\nSuccess Rates:")
    print(f"BUY success rate: {(buy_signals['strategy_success'] == 'Successful').mean():.2%}")
    print(f"SELL success rate: {(sell_signals['strategy_success'] == 'Successful').mean():.2%}")

    # Remove temporary columns before saving
    df_to_save = df.drop(columns=['next_close', 'price_diff'])

    # Save to CSV
    try:
        df_to_save.to_csv(output_csv, index=False)
        print(f"\nSuccessfully saved labeled data to '{output_csv}'")
    except Exception as e:
        print(f"\nError saving file '{output_csv}': {e}")

In [4]:
main()

Loaded 225 rows from ./data/gold_train.csv

First 5 rows of the dataset:
   Unnamed: 0            datetime        close  mvrv_btc_momentum  \
0           0 2025-02-24 00:00:00  95439.81388          -0.003825   
1           1 2025-02-24 00:20:00  95539.13964          -0.001923   
2           2 2025-02-24 00:30:00  95751.09609          -0.000972   
3           3 2025-02-24 00:40:00  95727.74231          -0.000021   
4           4 2025-02-24 01:00:00  95766.65282           0.000246   

   spot_volume_daily_sum  Signal  \
0             4592772971       1   
1             4599481100       1   
2             4621205570       1   
3             4619514085       1   
4             4620634709       1   

                                             summary  \
0  El debate sobre si el bitcoin puede convertirs...   
1  BlackRock's push for real-world asset tokeniza...   
2  The cryptocurrency market is experiencing vola...   
3  Bitcoin's price dipped below $96,000 amid mark...   
4  The cryptocu

/tmp/ipykernel_304645/118533575.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ True  True False  True False  True  True  True  True  True False  True
  True False  True False False False  True  True  True False False  True
 False False False False False False  True  True False  True False False
  True  True False False False  True  True  True False False False False
 False  True False False False  True  True  True  True False False  True
  True False False  True False False False False False False False False
  True  True False False  True False False  True  True  True  True  True
 False False  True  True False False False False False  True False False
  True False False  True False False False False False]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[buy_mask, 'strategy_success'] = df.loc[buy_mask, 'price_diff'] > 0
